# Social Network Analysis on trending Hashtags

## Crawling Twitter Data

In [ ]:
# Import 
import pandas as pd
import numpy as np
import tweepy
import matplotlib.pyplot as plt
import networkx as nx
from networkx.readwrite import json_graph
from plotly.offline import download_plotlyjs, init_notebook_mode,  iplot, plot
init_notebook_mode(connected=True)

In [ ]:
# Call twitter api keys
consumer_key = 'xxx'
consumer_secret = 'xxx'
access_token = 'xxx'
access_secret = 'xxx'
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth)

In [ ]:
# Search keywords
tweets = tweepy.Cursor(api.search,q="",tweet_mode="extended",since="2020-02-26",lang="id").items()
message,retweet_count,retweet,created_at,user_name,user_id=[],[],[],[],[],[]
count = 0
for tweet in tweets:
    count=count+1
    if hasattr(tweet, 'retweeted_status'):
        message.append(tweet.retweeted_status.full_text)
        retweet_count.append(tweet.retweet_count)
        retweet.append(tweet.retweeted_status.user.screen_name)
        created_at.append(tweet.created_at)
        user_name.append(tweet.user.screen_name)
        user_id.append(tweet.user.id)
    else:
        message.append(tweet.full_text)
        retweet_count.append(tweet.retweet_count)
        retweet.append(print(''))
        created_at.append(tweet.created_at)
        user_name.append(tweet.user.screen_name)
        user_id.append(tweet.user.id)
# insert tweets to database
for i in range(count):
    data=[message[i], retweet_count[i], retweet[i], created_at[i], user_name[i], user_id[i]]

In [ ]:
# make dataframe
df=pd.DataFrame({
    'author':retweet,
    'username':user_name,
    'retweet_count':retweet_count,
    'tweets':message
})
df = df.sort_values(['author'], ascending=[1]);
df[df['author'].isnull()]

In [ ]:
# Remove null account
# df=df.dropn()

## Matplotlib

In [ ]:
# Netwrokx
net = nx.from_pandas_edgelist(df, source="", target="")
# Plot it
nx.draw(G, with_labels=False, node_size=25)
plt.show()

## Plotly

In [ ]:
G = nx.convert_node_labels_to_integers(net, first_label=0, ordering='default', label_attribute=None)
pos=nx.fruchterman_reingold_layout(G)
#create labels
poslabs=nx.fruchterman_reingold_layout(net)
labels=list(poslabs) + list(' : ')
#create edges
Xe=[]
Ye=[]
for e in G.edges():
    Xe.extend([pos[e[0]][0], pos[e[1]][0], None])
    Ye.extend([pos[e[0]][1], pos[e[1]][1], None])
trace_edges=dict(type='scatter',
                 mode='lines',
                 x=Xe,
                 y=Ye,
                 line=dict(width=1, color='rgb(25,25,25)'),
                 hoverinfo='none' 
                )

#create nodes
Xn=[pos[k][0] for k in range(len(pos))]
Yn=[pos[k][1] for k in range(len(pos))]
trace_nodes=dict(type='scatter',
                 x=Xn, 
                 y=Yn,
                 mode='markers',
                 marker=dict(showscale=True,size=5,color=[],colorscale='Rainbow',reversescale=True,colorbar=dict(
                    thickness=15,
                    title='Node Connections',
                    xanchor='left',
                    titleside='right')),
                 text=labels,
                 hoverinfo='text')

#scale color by size
for node, adjacencies in enumerate(G.adjacency()):
    trace_nodes['marker']['color']+=tuple([len(adjacencies[1])])
#plot
axis=dict(showline=False, # hide axis line, grid, ticklabels and  title
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title='' 
          )
layout=dict(title= 'Social Network Analysis',  
            font= dict(family='Balto'),
            width=1000,
            height=1000,
            autosize=False,
            showlegend=False,
            xaxis=axis,
            yaxis=axis,
            margin=dict(l=40,r=40,b=85,t=100,pad=0,
            ),
            hovermode='closest',
#     plot_bgcolor='#000000',           
    )
fig = dict(data=[trace_edges,trace_nodes], layout=layout)
#run plot
iplot(fig)
plot(fig, filename='myplot.html')

## D3.JS

In [ ]:
# create degree
degrees = nx.degree()
ds = {}
for name, d in degrees: 
    ds[name] = d

nx.set_node_attributes(net, ds, 'degree')

In [ ]:
# Convert number to name
a=list(ds)
c=a+a
c.sort()

def Convert(lst): 
    res_dct = {lst[i]: lst[i + 1] for i in range(0, len(lst), 2)} 
    return res_dct 
d=Convert(c)
nx.set_node_attributes(net,d, 'name')

In [ ]:
# save as json file
data = json_graph.node_link_data(net)
with open('graph.json', 'w') as fp:
    json.dump(data, fp)